In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error

### Paths

In [38]:
main_path_energy = "/home/sebastian/github/SpecialProblem_Salo/drone_irrigation_vrep/src/data_base/paths_energy/"
main_path_total_energy = "/home/sebastian/github/SpecialProblem_Salo/drone_irrigation_vrep/src/data_base/paths_totals/"
main_path_turns = "/home/sebastian/Desktop/"

energy_path11_pd = pd.read_csv(main_path_energy + 'feat_pathv11_nuevo.csv')
energy_path12_pd = pd.read_csv(main_path_energy + 'feat_pathv12_nuevo.csv')
energy_path13_pd = pd.read_csv(main_path_energy + 'features_pathv13.csv')

energy_path_list = [energy_path11_pd,energy_path12_pd,energy_path13_pd]
complete_energy_pd = pd.concat(energy_path_list)

total_path11_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv11.csv')
total_path12_pd = pd.read_csv(main_path_total_energy + 'totalF_pathv12_nuevo.csv')
total_path13_pd = pd.read_csv(main_path_total_energy + 'totals_pathv13.csv')

total_path_list = [total_path11_pd,total_path12_pd,total_path13_pd]
complete_total_pd = pd.concat(total_path_list)

### Data split and preprocessing - complete_energy_pd - All data

this data comes from the folder "paths_energy"

In [39]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (2486, 622)


#### Fit Linear regression

In [40]:
linearRe = LinearRegression()
model = linearRe.fit(x_train,y_train)

#### Accuracy

In [41]:
predict = linearRe.predict(x_test)

r2 = r2_score(y_test,predict)
rmse = mean_squared_error(y_test,predict,squared=False)


print(f'r2 score: {r2}')
print(f'RMSE score: {rmse}')

r2 score: 0.6559967467411855
RMSE score: 1122109.8511876934


### Data split and preprocessing - complete_energy_pd - with eps

In [42]:
eps = 5
complete_energy_pd = complete_energy_pd[(complete_energy_pd['sim_point_dist']-complete_energy_pd['teo_point_dist']) < eps]
print(f'new shape of complete_energy_pd = {complete_energy_pd.shape}')

new shape of complete_energy_pd = (2577, 14)


In [43]:
X = complete_energy_pd.drop(columns=["missing_points","path_num","sim_drone_time","Energy","sim_point_dist","sim_Xdist","sim_Ydist","avg_error_target_dist","avg_energy"],axis=1)
y = complete_energy_pd["Energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler2 = StandardScaler()

x_train = scaler2.fit_transform(x_train)
x_test = scaler2.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (2061, 516)


#### Fit Linear regression

In [44]:
linearRe2 = LinearRegression()
model = linearRe2.fit(x_train,y_train)

#### Accuracy

In [45]:
predict = linearRe2.predict(x_test)

r2 = r2_score(y_test,predict)
rmse = mean_squared_error(y_test,predict,squared=False)


print(f'r2 score: {r2}')
print(f'RMSE score: {rmse}')

r2 score: 0.6940410269502381
RMSE score: 791905.3047954568


### Data split and preprocessing - complete_total_pd

this data comes from "path_totals" folder

In [47]:
X = complete_total_pd.drop(columns=["path_num","energy"],axis=1)
y = complete_total_pd["energy"].to_numpy()

X = X.to_numpy()

x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=35)

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

print(f'Train size/Test size {len(x_train),len(x_test)}')

Train size/Test size (466, 117)


#### Fit Linear regression

In [48]:
linearRe3 = LinearRegression()
model = linearRe3.fit(x_train,y_train)

#### Accuracy

In [49]:
predict = linearRe3.predict(x_test)

r2 = r2_score(y_test,predict)
rmse = mean_squared_error(y_test,predict,squared=False)


print(f'r2 score: {r2}')
print(f'RMSE score: {rmse}')

r2 score: -0.05646625577785702
RMSE score: 5357820.774397302
